In [26]:
import numpy as np
import matplotlib.pyplot as plt

from data_handler import DataHandler
from calculations import Calculations
from gaussian_nb import GaussianNB

dh = DataHandler("spambase.data")
data = dh.parse_data_no_header()
data = dh.shuffle_data(data)
data_train, data_test = dh.split_data(data)

tX, tY = dh.getXY(data_train, -1, -1)
mean, std = dh.zscore_data(tX) #Zscore is applied in Dynamic Split Fxn
tX = dh.apply_zscore(mean, std, tX)
vX, vY = dh.getXY(data_test, -1, -1)
vX = dh.apply_zscore(mean, std, vX)

gnb = GaussianNB()
means, xvars, priors = gnb.fit(tX, tY)
probs = gnb.getPFC(vX, vY, means, xvars, priors)
probs = np.array(probs)
classifiers = np.array(np.unique(vY))

preds = classifiers[np.argmax(probs, axis=1)]
preds = np.array([preds])
#Train Model into NP Array for Predictions
calc = Calculations(vY, preds)
acc, precision, recall, fmeasure  = calc.evaluate()
acc, precision, recall, fmeasure 

(0.8023483365949119,
 0.6607773851590106,
 0.9739583333333334,
 0.7873684210526316)

In [ ]:
# dh = DataHandler("CTG.csv")
# data = dh.parse_data()
# data = dh.shuffle_data(data)
# data_train, data_test = dh.split_data(data)
# tX, tY = dh.getXY(data_train, -1, -2)
# tY = np.delete(tY, 0, axis=1)
# mean, std = dh.zscore_data(tX)
# tX = dh.apply_zscore(mean, std, tX)
# vX = dh.apply_zscore(mean, std, vX)



# vX, vY = dh.getXY(data_test, -1, -2)
# vY = np.delete(vY, 0, axis=1)gnb = GaussianNB()
# split_X, priors = gnb.fit(tX, tY)
# vX_, vpriors = gnb.fit(vX, vY)

# means, xvars = gnb.mean_variance(split_X, tY)
# probs, preds = gnb.getPFC(vX, vY, means, xvars, vpriors)
# print(vY)
# preds = np.array([preds], dtype=float)
# print(preds)
# #Train Model into NP Array for Predictions
# calc2 = Calculations(vY, preds)
# TP, TN, FP, FN = calc2.setup_multi()
# calc2.accuracy(TP, TN, FP, FN)